In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

!pip install tensorflow-gpu==2.11 tensorflow_gnn
!pip install spektral
!pip install kneed

from google.colab import drive
from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
import tensorflow as tf
import tensorflow_gnn as tfgnn
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from spektral.layers import GCNConv
from spektral.utils import normalized_adjacency, add_self_loops
from kneed import KneeLocator
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from astropy.cosmology import Planck18 as cosmo
import pandas as pd


In [ ]:
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/FS2_Marguerite_positive_new.fits'

Mounted at /content/drive


In [ ]:
#Open selected data
with fits.open(file_path, memmap=True) as hdul:
    table = Table(hdul[1].data)

    #Chose only some columns from the table
    selected_table = table[['OBJECT_ID', 'RIGHT_ASCENSION', 'DECLINATION',
                            'PHZ_MEDIAN', 'Z_SPE','FLUX_VIS_APER',
                            'FLUXERR_VIS_APER', 'FLUX_Y_TOTAL_UNIF',
                            'FLUXERR_Y_TOTAL_UNIF','FLUX_J_TOTAL_UNIF',
                            'FLUXERR_J_TOTAL_UNIF','FLUX_H_TOTAL_UNIF',
                            'FLUXERR_H_TOTAL_UNIF',]]


#Select data inside a square (0.8ra x 0.8dec)
rad_min, rad_max = 148.0, 148.8
dec_min, dec_max = 1.5,2.3


filtered_data = table[(table['RIGHT_ASCENSION'] >= rad_min) &
                     (table['RIGHT_ASCENSION'] <= rad_max) &
                     (table['DECLINATION'] >= dec_min) &
                     (table['DECLINATION'] <= dec_max)&
                     (table['Z_SPE'] >= 0.0029)]

print(f"Filtered data: {len(filtered_data)}")

Dati filtrati: 160490


In [ ]:
#Compute colors YJ, JH and magnitudes H,Y,J

flux_table = filtered_data[['OBJECT_ID','RIGHT_ASCENSION', 'DECLINATION',
                            'PHZ_MEDIAN','Z_SPE', 'FLUX_VIS_APER',
                            'FLUX_Y_TOTAL_UNIF','FLUX_J_TOTAL_UNIF',
                            'FLUX_H_TOTAL_UNIF']]

colorYJ = []
colorJH = []
magnitudeH = []
magnitudeY  = []
magnitudeJ  = []
number = 0

for i in range(len(flux_table)):
    fluxY = flux_table['FLUX_Y_TOTAL_UNIF'][i]
    fluxJ = flux_table['FLUX_J_TOTAL_UNIF'][i]
    fluxH = flux_table['FLUX_H_TOTAL_UNIF'][i]

    if fluxY > 0 and fluxJ > 0 and fluxH > 0:
        x = -2.5 * np.log10(fluxY / fluxJ)
        y = -2.5 * np.log10(fluxJ / fluxH)
        mH = -2.5 * np.log10(fluxH) + 23.9
        mY = -2.5 * np.log10(fluxY) + 23.9
        mJ = -2.5 * np.log10(fluxJ) + 23.9
    else:
        x = 0
        y = 0
        mH = 0
        mY = 0
        mJ = 0
        number = number+1

    colorYJ.append(x)
    colorJH.append(y)
    magnitudeH.append(mH)
    magnitudeY.append(mY)
    magnitudeJ.append(mJ)

new_column = Column(colorYJ, name='COLOR Y-J')
new_column1 = Column(colorJH, name='COLOR J-H')
new_column2 = Column(magnitudeH, name='MAGNITUDE H')
new_column3 = Column(magnitudeY, name='MAGNITUDE Y')
new_column4 = Column(magnitudeJ, name='MAGNITUDE J')

#Append new columns for colors and magnitudes to the existing table
flux_table.add_column(new_column)
flux_table.add_column(new_column1)
flux_table.add_column(new_column2)
flux_table.add_column(new_column3)
flux_table.add_column(new_column4)

print(f'Number of data with flux minor or equal to zero: {number}')
print(len(flux_table))

Numero dati con flusso minore o uguale a zero: 685
160490


In [ ]:
#Delete data with negative fluxes

filtered_table = flux_table[(flux_table['COLOR Y-J'] != 0) &
                            (flux_table['COLOR J-H'] != 0)]
print(len(filtered_table))

159805


In [ ]:
#Use K nearest neighbors to create the adjacency matrix

galaxies = np.stack([
    filtered_table['RIGHT_ASCENSION'],
    filtered_table['DECLINATION'],
    filtered_table['COLOR Y-J'],
    filtered_table['COLOR J-H'],
    filtered_table['MAGNITUDE H']
], axis=1).astype(float)

#Scale data
scaler = StandardScaler()
galaxies_scaled = scaler.fit_transform(galaxies)

k = 40

features = tf.constant(galaxies_scaled, dtype=tf.float32)
adjacency_matrix = kneighbors_graph(galaxies_scaled, n_neighbors=k,
                                    mode='distance',
                                    include_self=False)

#Normalize adjacency matrix between 0 and 1
min_val = adjacency_matrix.data.min()
max_val = adjacency_matrix.data.max()
adjacency_matrix.data = (adjacency_matrix.data - min_val) / (max_val - min_val)

#Set closest points closer to 1
adjacency_matrix.data = 1 - adjacency_matrix.data

#Convert into appriopriate format
adjacency_matrix_coo = adjacency_matrix.tocoo()

#Create the graph with edges and weights
edges = tf.constant(adjacency_matrix_coo.nonzero(), dtype=tf.int64)
weights = tf.constant(adjacency_matrix_coo.data, dtype=tf.float32)

graph = tfgnn.GraphTensor.from_pieces(
    node_sets = {
        "nodes": tfgnn.NodeSet.from_fields(
            features={"features": features},
            sizes=[len(features)]
        )
    },
    edge_sets={
        "edges": tfgnn.EdgeSet.from_fields(
            features={"weights": weights},
            sizes=[edges.shape[1]],
            adjacency=tfgnn.Adjacency.from_indices(
                source=("nodes", edges[:, 0]),
                target=("nodes", edges[:, 1])
            )
        )
    }
)


In [ ]:
#K-means clustering classification with scaled data
RA = galaxies_scaled[:,0]
DEC = galaxies_scaled[:,1]
ColorY_J = galaxies_scaled[:,2]
ColorJ_H = galaxies_scaled[:,3]
MagnitudeH = galaxies_scaled[:,4]


coordinates = np.column_stack((ColorY_J, ColorJ_H, MagnitudeH, RA, DEC))


#Try with different values of k (number of clusters)
clusterNum = range(5, 30)
distance = []

for i in clusterNum:
    k_means = KMeans(init = "k-means++", n_clusters = i, n_init = 12)
    k_means.fit(coordinates)
    #Save inertia (sum quadratic distances intra-cluster)
    distance.append(k_means.inertia_)


In [ ]:
#Find elbow point (best number of k)
knee_locator = KneeLocator(clusterNum, distance, curve="convex", direction="decreasing")

elbow_point = knee_locator.knee
print(f"Elbow point: {elbow_point}")

Elbow point: 12


In [ ]:
#Faccio K-Means con l'elbow point, nota che non vogliamo che le distanze tra i punti di un cluster siano troppo alte, ma non devono neanche
#essere troppo basse
#K-means using elbow point

clusterNum = elbow_point

k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(coordinates)
labels = k_means.labels_ #dice ogni elemento del dataset a quale cluster è stato assegnato
cluster_centers = k_means.cluster_centers_

#Count number of elements in each cluster
unique, counts = np.unique(labels, return_counts=True)

for cluster, count in zip(unique, counts):
    print(f"Cluster {cluster}: {count} elements")

print(cluster_centers)

Cluster 0: 5963 elementi
Cluster 1: 21563 elementi
Cluster 2: 9807 elementi
Cluster 3: 20807 elementi
Cluster 4: 9195 elementi
Cluster 5: 11842 elementi
Cluster 6: 24416 elementi
Cluster 7: 2910 elementi
Cluster 8: 20011 elementi
Cluster 9: 10920 elementi
Cluster 10: 9358 elementi
Cluster 11: 13013 elementi
[[-1.32866029e+00  2.48824076e+00  7.50171515e-01 -8.60308728e-03
  -2.33107581e-02]
 [-4.39579391e-01 -3.25582734e-01  2.84239193e-01 -9.24927719e-01
   9.12652112e-01]
 [ 3.85042851e-02  1.54083188e-01 -2.02605634e+00 -9.31285529e-01
   1.70313154e-01]
 [-4.68523869e-01 -3.31036862e-01  3.61272421e-01  9.42553524e-01
  -8.38758114e-01]
 [-1.45200770e-02 -1.65483141e+00  9.67795130e-01 -2.17656502e-01
  -2.01240695e-01]
 [ 9.87985640e-01  7.47687297e-01  2.72667298e-01 -6.27665392e-01
   7.02206840e-01]
 [-1.69169563e-01 -7.02996195e-02  1.76746617e-01 -9.28506521e-01
  -9.17144849e-01]
 [ 3.90874580e+00  6.40750685e-01  6.93483413e-01 -2.10169710e-02
  -1.08092815e-01]
 [-2.622264

In [ ]:
#Silhouette Score
silhouette_avg = silhouette_score(coordinates, labels)

print(f"Mean Silhouette Score: {silhouette_avg}")

Silhouette Score medio: 0.1774513872135572


In [ ]:
#Calinski-Harabasz Score
ch_score = calinski_harabasz_score(coordinates, labels)
print(f"Calinski-Harabasz Score: {ch_score}")

Calinski-Harabasz Score: 22741.616057555573


In [ ]:
#Check that values of adjacency matrix are normalized between 0 and 1

values = adjacency_matrix.data

#Find values out of range
out_of_range_values = values[(values < 0) | (values > 1)]

if len(out_of_range_values) > 0:
    print("Values out of the interval [0, 1]:", out_of_range_values)
    print("Number of values out of the interval:", len(out_of_range_values))
else:
    print("All values are in interval [0, 1].")

print(adjacency_matrix_coo)

Tutti i valori sono nell'intervallo [0, 1].
  (0, 2)	0.9965175987893231
  (0, 59670)	0.9844804691995199
  (0, 117451)	0.9840879396177237
  (0, 114683)	0.9813127114219512
  (0, 117408)	0.9805308197404992
  (0, 58223)	0.9793239231022838
  (0, 6786)	0.977265406022681
  (0, 59003)	0.9764323105538446
  (0, 5729)	0.9763910906430702
  (0, 116434)	0.9757159237304871
  (0, 4184)	0.9756090670653011
  (0, 10658)	0.9753682151572521
  (0, 58057)	0.9742078859057891
  (0, 61224)	0.973963377915438
  (0, 59626)	0.9736632628403089
  (0, 3372)	0.973445836169668
  (0, 120904)	0.9729861597698789
  (0, 57315)	0.9724657589263558
  (0, 273)	0.9717129393168255
  (0, 63964)	0.9715974634592234
  (0, 95)	0.9713230250969518
  (0, 5123)	0.9709516987190536
  (0, 2452)	0.970935024481945
  (0, 3865)	0.9705558493091656
  (0, 58048)	0.9700831913896439
  :	:
  (159804, 136693)	0.9768944378596249
  (159804, 89181)	0.976802913459789
  (159804, 134775)	0.9762653495918274
  (159804, 44517)	0.9761158897619673
  (159804, 56617

In [ ]:
print("Shape after scaling:", galaxies_scaled.shape)
print("Mean of every column (after scaling):", galaxies_scaled.mean(axis=0))
print("Standard deviation of every column:", galaxies_scaled.std(axis=0))
print("Adjacency matrix shape:", adjacency_matrix_coo.shape)
print("Adjacency matrix type:", adjacency_matrix_coo.dtype)

Forma dopo lo scaling: (159805, 5)
Media di ogni colonna (dopo lo scaling): [-2.23092059e-14  2.58501577e-16 -8.89660292e-16 -3.38278123e-16
  4.62869940e-11]
Deviazione standard di ogni colonna: [1. 1. 1. 1. 1.]
Forma della matrice di adiacenza: (159805, 159805)
Tipo matrice di Adiacenza: float64


In [ ]:
#Check some features of the graph

num_nodes = graph.node_sets["nodes"].sizes[0]
num_edges = graph.edge_sets["edges"].sizes[0]
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

num_nodes = tf.cast(num_nodes, tf.int64)
num_edges = tf.cast(num_edges, tf.int64)

max_edges = num_nodes * (num_nodes - 1) // 2
density = num_edges / max_edges
print(f"Graph density: {density}")
print(f"Maximum number of edges: {max_edges}")

Numero di nodi: 159805
Numero di archi: 6392199
Densità del grafo: 0.0005006131729164917
Numero massimo di connessioni: 12768739110


In [ ]:
#Create sparse matrix with tf.sparse from Tensorflow

n_nodes = galaxies_scaled.shape[0]
n_features = galaxies_scaled.shape[1]


"""
Note: adjacency_matrix_coo was already a sparse matrix, but creating it using
tf.sparse makes it readable for future operations using Tensorflow.
"""

adj = tf.sparse.SparseTensor(
    indices=np.vstack((adjacency_matrix_coo.row, adjacency_matrix_coo.col)).T,
    values=adjacency_matrix_coo.data,
    dense_shape=adjacency_matrix_coo.shape

)

#Add self loops to the adjacency matrix
self_loops = tf.sparse.SparseTensor(
    indices=[[i, i] for i in range(adj.shape[0])],
    values=tf.ones(adj.shape[0], dtype=adj.dtype),
    dense_shape=adj.shape
)


adj_with_self_loops = tf.sparse.add(adj, self_loops)


features = tf.convert_to_tensor(galaxies_scaled, dtype=tf.float32)

In [ ]:
print(features)

tf.Tensor(
[[ 1.1664892  -1.5366001  -0.84484017 -0.9340779  -3.151268  ]
 [ 1.1775665  -1.5447432  -0.7698594  -0.87228596 -2.2857234 ]
 [ 1.2028859  -1.524551   -0.80394244 -0.8879684  -3.0880866 ]
 ...
 [ 1.565006    1.2685736  -0.46795356  1.7995236   0.9867769 ]
 [ 1.511466    1.3483487   0.8669721   1.99705     0.79046714]
 [ 1.3352852   1.7367977  -0.963896    1.4116533   0.27871838]], shape=(159805, 5), dtype=float32)


In [ ]:
print(adj_with_self_loops)

SparseTensor(indices=tf.Tensor(
[[     0      0]
 [     0      2]
 [     0  59670]
 ...
 [159804 134808]
 [159804  41226]
 [159804 159804]], shape=(6552005, 2), dtype=int64), values=tf.Tensor([1.         0.9965176  0.98448047 ... 0.97140694 0.97140683 1.        ], shape=(6552005,), dtype=float64), dense_shape=tf.Tensor([159805 159805], shape=(2,), dtype=int64))
tf.Tensor(0.0, shape=(), dtype=float64)


In [ ]:
#Create the GAE model

class GraphAutoencoder(Model):
    def __init__(self, hidden_dim, latent_dim):
        super(GraphAutoencoder, self).__init__()
        self.gcn1 = GCNConv(hidden_dim, activation="relu")
        self.gcn2 = GCNConv(hidden_dim // 2, activation="relu")
        self.gcn3 = GCNConv(latent_dim, activation=None)

    def encoder(self, x, adj):
        #Apply GCN layers to obtain embeddings
        z = self.gcn1([x, adj])
        z = self.gcn2([z, adj])
        z = self.gcn3([z, adj])
        #Normalize features vector to mean = 0 and dev_std = 1
        z = tf.nn.l2_normalize(z, axis=1)
        return z

    def decoder(self, z, adj_indices):
        """
        Reconstruct the adjacency matrix with inner product.
        Calculates the inner product only for the specified indices
        (sparse matrix), the highest the inner product the most similar
        are two vectors (elements of z).
        """
        reconstructed_values = tf.reduce_sum(
            tf.gather(z, adj_indices[:, 0]) * tf.gather(z, adj_indices[:, 1]),
            axis=1)

        #Normalize between 0 and 1, as for the initial adjacency matrix
        min_val = tf.reduce_min(reconstructed_values)
        max_val = tf.reduce_max(reconstructed_values)
        reconstructed_values = (reconstructed_values - min_val) / (max_val - min_val)

        #Create a SparseTensor for reconstructed matrix with tf.sparse
        reconstructed_adj = tf.sparse.SparseTensor(
            indices=adj_indices,
            values=reconstructed_values,
            dense_shape=adj.shape
        )
        return reconstructed_adj


    def call(self, inputs):
        x, adj = inputs
        z = self.encoder(x, adj)
        reconstructed_adj = self.decoder(z, adj.indices)
        return reconstructed_adj


In [ ]:
def reconstruction_loss(y_true, y_pred):
    """
    Loss function between A (original values) and A_hat (predicted values
    from the model).
    """
    A = y_true.values
    A_reconstructed = y_pred.values

    #Limit all values between 0 and 1 in case some of them are too small
    A_reconstructed = tf.clip_by_value(A_reconstructed, 1e-6, 1 - 1e-6)  #Avoid log(0)

    #Convert A to A_reconstructed format (32 bit)
    A = tf.cast(A, dtype=A_reconstructed.dtype)

    #Mean squared error loss function
    mse_loss = tf.reduce_mean(tf.square(A - A_reconstructed))

    return mse_loss

In [ ]:
#Model parameter
hidden_dim = 64 #capture high information
latent_dim = 3 #force clusters formation

#Initialize the model
gae = GraphAutoencoder(hidden_dim=hidden_dim, latent_dim=latent_dim)

#Set optimizer to the initial value of the learning rate
init_learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(init_learning_rate)

#Set hyperparameters
lr_decay_factor = 0.5
lr_patience = 5
min_lr = 1e-5
#set initial best_loss to infinite so first loss will be less
best_loss = float("inf")
no_improvement_count = 0
min_delta = 1e-3


#Training loop
epochs = 20
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        #Pass data (features) to the model
        reconstructed_adj = gae([features, adj_with_self_loops])
        negative_values = tf.math.less(reconstructed_adj.values, 0)

        #Count how many values are less than zero
        count_negative = tf.reduce_sum(tf.cast(negative_values, tf.int32))
        print(f"Number of values less than zero: {count_negative.numpy()}")

        #Compute the loss
        loss = reconstruction_loss(adj_with_self_loops, reconstructed_adj)

    #Apply gradients
    weights_before = [tf.identity(w) for w in gae.trainable_variables]
    gradients = tape.gradient(loss, gae.trainable_variables)
    optimizer.apply_gradients(zip(gradients, gae.trainable_variables))
    weights_after = gae.trainable_variables

    #Show weights difference after each epoch
    for i, (w_before, w_after) in enumerate(zip(weights_before, weights_after)):
        print(f"Layer {i}: Weights difference = {tf.reduce_sum(tf.abs(w_after - w_before)).numpy():.6f}")

    #Update learning rate after 5 epochs of non improvment in loss
    if best_loss - loss > min_delta:
        best_loss = loss
        no_improvement_count = 0  #Reset if loss improves
    else:
        no_improvement_count += 1
    #Update optimizer
    if no_improvement_count >= lr_patience:
        new_lr = max(optimizer.learning_rate.numpy() * lr_decay_factor, min_lr)
        optimizer.learning_rate.assign(new_lr)
        print(f"Learning rate reduced to {new_lr:.6f}")
        #Reset counting
        no_improvement_count = 0

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.numpy():.6f}, LR: {optimizer.learning_rate.numpy():.6f}\n")

Numero di valori minori di zero: 0
Layer 0: Differenza nei pesi = 3.127844
Layer 1: Differenza nei pesi = 0.525357
Layer 2: Differenza nei pesi = 18.365776
Layer 3: Differenza nei pesi = 0.085465
Layer 4: Differenza nei pesi = 0.910976
Layer 5: Differenza nei pesi = 0.001736
Epoch 1/20, Loss: 0.002336, LR: 0.010000

Numero di valori minori di zero: 0
Layer 0: Differenza nei pesi = 2.046196
Layer 1: Differenza nei pesi = 0.322843
Layer 2: Differenza nei pesi = 12.756344
Layer 3: Differenza nei pesi = 0.057811
Layer 4: Differenza nei pesi = 0.631131
Layer 5: Differenza nei pesi = 0.001172
Epoch 2/20, Loss: 0.000425, LR: 0.010000

Numero di valori minori di zero: 0
Layer 0: Differenza nei pesi = 1.602645
Layer 1: Differenza nei pesi = 0.250523
Layer 2: Differenza nei pesi = 10.180656
Layer 3: Differenza nei pesi = 0.044903
Layer 4: Differenza nei pesi = 0.492020
Layer 5: Differenza nei pesi = 0.000905
Epoch 3/20, Loss: 0.000313, LR: 0.010000

Numero di valori minori di zero: 0
Layer 0: Di

In [ ]:
#Show latent embedding layer z (features tensor after the model was applied)
z = gae.encoder(features, adj_with_self_loops)
z_numpy = z.numpy()
print("Shape di z_numpy (NumPy):", z_numpy.shape)
print(z_numpy)

Shape di z_numpy (NumPy): (159805, 3)
[[ 0.5199357  -0.39485395  0.7574675 ]
 [ 0.51171243 -0.29608056  0.80652744]
 [ 0.51994675 -0.39086345  0.7595269 ]
 ...
 [ 0.31650653 -0.65099126  0.6899521 ]
 [ 0.36073747 -0.7535651   0.5495528 ]
 [ 0.3076424  -0.63321763  0.7102052 ]]


In [ ]:
#Keep correspondence between z_numpy and initial features (in rad and dec).
#Create z matrix with additional features for every object (object ra and dec)
ra_dec_array = np.array([filtered_table['RIGHT_ASCENSION'],
                         filtered_table['DECLINATION']]).T
z_numpy_with_ra_dec = np.concatenate([z_numpy, ra_dec_array], axis=1)
print(z_numpy_with_ra_dec)

[[  0.5199357   -0.39485395   0.7574675  148.6701       1.5350571 ]
 [  0.51171243  -0.29608056   0.80652744 148.67267      1.5331802 ]
 [  0.51994675  -0.39086345   0.7595269  148.67853      1.5378342 ]
 ...
 [  0.31650653  -0.65099126   0.6899521  148.76233      2.1816025 ]
 [  0.36073747  -0.7535651    0.5495528  148.74994      2.1999893 ]
 [  0.3076424   -0.63321763   0.7102052  148.70917      2.2895203 ]]


In [ ]:
#K-means algorithm on the embedding features, after the model was applied

range_k = range(5, 30)  #try with different k
inertias = []

for i in range_k:
    kmeans = KMeans(init = "k-means++", n_clusters = i, n_init = 12)
    kmeans.fit(z_numpy)
    inertias.append(kmeans.inertia_)

In [ ]:
#Find elbow point
knee_locator = KneeLocator(range_k, inertias, curve="convex", direction="decreasing")
print("Best number of clusters:", knee_locator.knee)

Best number of clusters: 12


In [ ]:
kmeans = KMeans(n_clusters=knee_locator.knee, n_init=12)
kmeans.fit(z_numpy)

#Show results
centri_clusters = kmeans.cluster_centers_

print("Clusters centers:")
print(centri_clusters)

cluster_labels = kmeans.labels_
distances = kmeans.transform(z_numpy)

#Count elements for each cluster
n_elementi_per_cluster = np.bincount(cluster_labels)

print("Number of elements for every cluster:")
for cluster, count in enumerate(n_elementi_per_cluster):
    print(f"Cluster {cluster}: {count} elements")

Centri dei cluster trovati:
[[ 0.3312029  -0.4951792   0.79972935]
 [ 0.23669782 -0.8699901   0.4220965 ]
 [ 0.5032504  -0.539619    0.669855  ]
 [ 0.47849643 -0.36936164  0.7926298 ]
 [ 0.19185047 -0.77162033  0.59920776]
 [ 0.36788705 -0.22134659  0.8986001 ]
 [ 0.17551951 -0.59700364  0.7776899 ]
 [ 0.43374655 -0.82213485  0.35596022]
 [ 0.30173084 -0.92494345  0.20843914]
 [ 0.23098357 -0.35827377  0.8994949 ]
 [ 0.46903893 -0.70299745  0.5264753 ]
 [ 0.3463828  -0.6467362   0.67504513]]
Numero di elementi per cluster:
Cluster 0: 16049 elementi
Cluster 1: 11513 elementi
Cluster 2: 15207 elementi
Cluster 3: 13885 elementi
Cluster 4: 11122 elementi
Cluster 5: 13168 elementi
Cluster 6: 11702 elementi
Cluster 7: 12232 elementi
Cluster 8: 11061 elementi
Cluster 9: 11679 elementi
Cluster 10: 17231 elementi
Cluster 11: 14956 elementi


In [ ]:
"""
Apply random reduction of the elements of every cluster using the median
absolute deviation to reduce the influence of outliers and standard deviations
afterwards.
"""

#Create a list with only three features and another adding ra and dec, but
#keeping correspondence in index
filtered_clusters = []
filtered_clusters_with_ra_dec = []

for cluster in np.unique(cluster_labels):
    cluster_indices = np.where(cluster_labels == cluster)[0]
    cluster_distances = distances[cluster_indices, cluster]
    distances_median = np.median(cluster_distances)

    #Apply MAD - Median Absolute Deviation
    mad = np.median(np.abs(cluster_distances - distances_median))
    cluster_threshold_upper_mad = distances_median + mad
    cluster_threshold_lower_mad = distances_median - mad

    #Determine initial inliers using MAD
    cluster_indices_inliers = np.where(
        (cluster_distances <= cluster_threshold_upper_mad)
        & (cluster_distances >= cluster_threshold_lower_mad))[0]

    #Determine initial outliers using MAD
    cluster_indices_outliers = np.where(
        (cluster_distances > cluster_threshold_upper_mad) |
        (cluster_distances < cluster_threshold_lower_mad))[0]

    #Keep deleting outliers from every cluster until they are less than 10
    while len(cluster_indices_outliers) > 10:
        cluster_distances_new = distances[cluster_indices_inliers, cluster]
        distances_mean = np.mean(cluster_distances_new)
        #Use standard deviation from now on
        std_dev = np.std(cluster_distances_new)
        cluster_threshold_upper_std = distances_mean + 2*std_dev
        cluster_threshold_lower_std = distances_mean - 2*std_dev

        cluster_indices_inliers = np.where(
            (cluster_distances_new <= cluster_threshold_upper_std)
            & (cluster_distances_new >= cluster_threshold_lower_std))[0]

        cluster_indices_outliers = np.where(
            (cluster_distances_new > cluster_threshold_upper_std)
            | (cluster_distances_new < cluster_threshold_lower_std))[0]

    filtered_clusters.append(z_numpy[cluster_indices_inliers])
    filtered_clusters_with_ra_dec.append(z_numpy_with_ra_dec[cluster_indices_inliers])

filtered_clusters = np.vstack(filtered_clusters)
filtered_clusters_with_ra_dec = np.vstack(filtered_clusters_with_ra_dec)


In [ ]:
#K-means with filtered_clusters

range_k = range(5, 30)
inertias = []

for i in range_k:
    kmeans = KMeans(init = "k-means++", n_clusters = i, n_init = 12)
    kmeans.fit(filtered_clusters)
    inertias.append(kmeans.inertia_)

In [ ]:
#Find elbow point
knee_locator = KneeLocator(range_k, inertias, curve="convex", direction="decreasing")
print("Best number of clusters:", knee_locator.knee)

Best number of clusters: 12


In [ ]:
kmeans = KMeans(n_clusters=knee_locator.knee, n_init=12)
kmeans.fit(filtered_clusters)

#Show results
centri_clusters_post = kmeans.cluster_centers_

print("Cluster centers:")
print(centri_clusters)

cluster_labels_post = kmeans.labels_

#Count number of elements for each cluster
n_elementi_per_cluster = np.bincount(cluster_labels_post)

print("Number of elements in every cluster:")
for cluster, count in enumerate(n_elementi_per_cluster):
    print(f"Cluster {cluster}: {count} elements")

Centri dei cluster trovati:
[[ 0.3312029  -0.4951792   0.79972935]
 [ 0.23669782 -0.8699901   0.4220965 ]
 [ 0.5032504  -0.539619    0.669855  ]
 [ 0.47849643 -0.36936164  0.7926298 ]
 [ 0.19185047 -0.77162033  0.59920776]
 [ 0.36788705 -0.22134659  0.8986001 ]
 [ 0.17551951 -0.59700364  0.7776899 ]
 [ 0.43374655 -0.82213485  0.35596022]
 [ 0.30173084 -0.92494345  0.20843914]
 [ 0.23098357 -0.35827377  0.8994949 ]
 [ 0.46903893 -0.70299745  0.5264753 ]
 [ 0.3463828  -0.6467362   0.67504513]]
Numero di elementi per cluster:
Cluster 0: 920 elementi
Cluster 1: 527 elementi
Cluster 2: 837 elementi
Cluster 3: 440 elementi
Cluster 4: 677 elementi
Cluster 5: 753 elementi
Cluster 6: 444 elementi
Cluster 7: 665 elementi
Cluster 8: 657 elementi
Cluster 9: 480 elementi
Cluster 10: 429 elementi
Cluster 11: 632 elementi


In [ ]:
#Silhouette Score before and after filtering
silhouette_avg = silhouette_score(z_numpy, cluster_labels)

print(f"Mean Silhouette Score: {silhouette_avg}")


silhouette_avg_post = silhouette_score(filtered_clusters, cluster_labels_post)

print(f"Mean Silhouette Score after filtering: {silhouette_avg_post}")



Silhouette Score medio: 0.3427370488643646
Silhouette Score medio dopo filtraggio: 0.3772273063659668


In [ ]:
#Calinski Harabasz Score before and after flitering
ch_score = calinski_harabasz_score(z_numpy, cluster_labels)
print(f"Calinski-Harabasz Score: {ch_score}")

ch_score_post = calinski_harabasz_score(filtered_clusters, cluster_labels_post)
print(f"Calinski-Harabasz Score after filtering: {ch_score_post}")

Calinski-Harabasz Score: 174857.97040454394
Calinski-Harabasz Score: 8519.099876431039


In [ ]:
cluster_centers_and_radia = []

#Find RA and DEC of every cluster center
for cluster_label in np.unique(cluster_labels_post):
    #Get data for the current cluster
    cluster_data = filtered_clusters_with_ra_dec[cluster_labels_post == cluster_label]
    ra_dec_data = cluster_data[:, -2:]  #Pick only last two columns (Ra and Dec)

    #Find cluster center as (ra,dec)
    center = np.mean(ra_dec_data, axis=0)

    #Compute max and mean radius for every cluster
    distances = np.sqrt(np.sum((ra_dec_data - center) ** 2, axis=1))
    radius_max = np.max(distances)
    radius_mean = np.mean(distances)

    #List with centers and radia
    cluster_centers_and_radia.append((center, radius_mean, radius_max))

    print(f"Cluster number:, {cluster_label:.5f}, center: [{center[0]:.5f},{center[1]:.5f}]")
    print(f"maximum radius: {radius_max:.5f}, mean radius: {radius_mean:.5f}.\n")



Cluster number:, 0, center: [148.2943      1.7421131]
maximum radius: 0.7027941942214966, mean radius: 0.27212727069854736.

Cluster number:, 1, center: [148.21521     2.0762205]
maximum radius: 0.5808811187744141, mean radius: 0.19195765256881714.

Cluster number:, 2, center: [148.60046     1.6382128]
maximum radius: 0.3103255033493042, mean radius: 0.12304067611694336.

Cluster number:, 3, center: [148.23235    1.867824]
maximum radius: 0.6622401475906372, mean radius: 0.2915003001689911.

Cluster number:, 4, center: [148.3513     1.723611]
maximum radius: 0.692415714263916, mean radius: 0.2910735607147217.

Cluster number:, 5, center: [148.62563     1.7211549]
maximum radius: 0.29718953371047974, mean radius: 0.15579475462436676.

Cluster number:, 6, center: [148.26538    2.141495]
maximum radius: 0.4321249723434448, mean radius: 0.18252351880073547.

Cluster number:, 7, center: [148.55695     1.9382267]
maximum radius: 0.4998573064804077, mean radius: 0.16647346317768097.

Cluster 

In [ ]:
print(cluster_centers_and_radia)

[(array([148.2943   ,   1.7421131], dtype=float32), 0.27212727, 0.7027942), (array([148.21521  ,   2.0762205], dtype=float32), 0.19195765, 0.5808811), (array([148.60046  ,   1.6382128], dtype=float32), 0.123040676, 0.3103255), (array([148.23235 ,   1.867824], dtype=float32), 0.2915003, 0.66224015), (array([148.3513  ,   1.723611], dtype=float32), 0.29107356, 0.6924157), (array([148.62563  ,   1.7211549], dtype=float32), 0.15579475, 0.29718953), (array([148.26538 ,   2.141495], dtype=float32), 0.18252352, 0.43212497), (array([148.55695  ,   1.9382267], dtype=float32), 0.16647346, 0.4998573), (array([148.53433  ,   1.6848291], dtype=float32), 0.20371322, 0.4878445), (array([148.31464  ,   1.8653436], dtype=float32), 0.2698749, 0.61817455), (array([148.15324  ,   2.1415954], dtype=float32), 0.17323437, 0.4450561), (array([148.39345  ,   2.0881155], dtype=float32), 0.22231698, 0.45605248)]


In [ ]:
file_path1 = '/content/drive/My Drive/galaxy_clusters.FS2.1_halos_with_true_richness_30deg2_ngal10.txt'

In [ ]:
df = pd.read_csv(file_path1, sep='\s+')
df.columns = df.columns.str.strip()
radia_z_data_true = df[[ 'rs_halo', 'rvir_halo', 'true_redshift_gal']].values
ra_dec_data_true = df[['ra_gal', 'dec_gal']].values

#Print some useful information
print("Column names after stripping:", df.columns.tolist())
print("Shape of ra_dec_data:", ra_dec_data_true.shape)
print("Minimum of ra:", ra_dec_data_true[:,0].min())
print("Maximum of ra:", ra_dec_data_true[:,0].max())
print("Minimum of dec:", ra_dec_data_true[:,1].min())
print("Maximum of dec:", ra_dec_data_true[:,1].max())

Column names after stripping: ['halo_id', 'galaxy_id', 'kind', 'lm_halo', 'n_sats_halo', 'rs_halo', 'rvir_halo', 'ra_gal', 'dec_gal', 'true_redshift_gal', 'observed_redshift_gal', 'euclid_vis', 'euclid_nisp_y', 'euclid_nisp_j', 'euclid_nisp_h', 'lsst_g', 'lsst_i', 'lsst_r', 'lsst_y', 'lsst_z', 'mw_extinction', 'richness_mag24', 'richness_true_dm1.5', 'richness_true_dm2.0']
Shape of ra_dec_data: (15766, 2)
Minimum of ra: 146.00024
Maximum of ra: 151.49976
Minimum of dec: 1.0031204
Maximum of dec: 6.499687


In [ ]:
#Find matching clusters (centers)
matching_centers = []
matching_radia = []
matched_centers = []
matched_centers_radia_z = []

#Create a structured array with fixed tipes
dtype = [('centers', float, 2), ('radia_mean', float), ('radia_max', float)]
cluster_centers_and_radia = np.array(cluster_centers_and_radia, dtype = dtype)

#Define constants
c = 299792
H0 = 70

for element in cluster_centers_and_radia:
    #Trova se il centro è presente nei dati, restituisce un array booleano con
    #valori True o False se la coppia di ra e dec corrisponde al centro
    #element[0] = centri trovati da me, element[1] = raggi media miei
    #element[2] = raggi massimi miei
    matches = np.isclose(ra_dec_data_true, element['centers'], atol=0.01).all(axis=1)
    #If there is correspondence
    if matches.any():
        matching_centers.append(element['centers'])
        matching_radia.append((element['radia_mean'], element['radia_max']))
        matched_centers.append(ra_dec_data_true[matches])
        matched_centers_radia_z.append(radia_z_data_true[matches])

#Show matching centers
if matching_centers:
    print("Centers that correspond to actual centers:\n")
    for center, radia, true_centers, radia_and_z in zip(matching_centers,
                                        matching_radia,matched_centers,
                                        matched_centers_radia_z):
        print(f"Center [{center[0]:.4f},{center[1]:.4f}] with mean radius: {radia[0]:.4f} e maximum: {radia[1]:.4f}, correspond to:")
        for row1, row2 in zip(true_centers, radia_and_z):
            distance = cosmo.comoving_distance(row2[2]).value*1000 #in kpac
            theta_rs_deg = np.degrees(row2[0] / distance)  #Raggio angolare in gradi
            theta_rvir_deg = np.degrees(row2[1]/distance)  #Raggio viriale in gradi
            print(f"[{row1[0]:.4f},{row1[1]:.4f}]")
            print(f"with scale radius {theta_rs_deg:.4f} degrees and virial radius {theta_rvir_deg:.4f} degrees.")
        print("\n\n")
    print(f"{len(matching_centers)} correspondig centers found out of {knee_locator.knee}")
else:
    print("No center correspond to the your found center.\n")

Centri che coincidono con i dati veri:

Centro [148.2943,1.7421] con raggio medio: 0.2721 e massimo: 0.7028, corrisponde a:
[148.2978,1.7388]
con raggio di scala 0.0031 gradi e raggio viriale 0.0092 gradi.



Centro [148.2323,1.8678] con raggio medio: 0.2915 e massimo: 0.6622, corrisponde a:
[148.2249,1.8756]
con raggio di scala 0.0025 gradi e raggio viriale 0.0201 gradi.



Centro [148.2654,2.1415] con raggio medio: 0.1825 e massimo: 0.4321, corrisponde a:
[148.2754,2.1398]
con raggio di scala 0.0028 gradi e raggio viriale 0.0161 gradi.



Centro [148.5569,1.9382] con raggio medio: 0.1665 e massimo: 0.4999, corrisponde a:
[148.5660,1.9482]
con raggio di scala 0.0026 gradi e raggio viriale 0.0158 gradi.
[148.5532,1.9397]
con raggio di scala 0.0027 gradi e raggio viriale 0.0086 gradi.



Centro [148.5343,1.6848] con raggio medio: 0.2037 e massimo: 0.4878, corrisponde a:
[148.5292,1.6831]
con raggio di scala 0.0044 gradi e raggio viriale 0.0143 gradi.



Centro [148.3146,1.8653] con ragg

Puoi provare a normalizzare z_numpy tra 0 e 1 prima di effettuare clustering, magari i raggi medio e massimo diminuiscono un po'.